In [7]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pymongo
import json
import datetime
import time
import re
from lxml import etree

In [4]:
#建立MongoDB客户端
mongo_client =pymongo.MongoClient(host='localhost',port=27017)  #连接本地的服务端
mongo_db=mongo_client.smartphone  #指定操作的数据库
mongo_coll_1 = mongo_db["T1_SMTPH_Zollfo_1"]
mongo_coll_2 = mongo_db["T2_SMTPH_Zollfo_2"]
mongo_coll_3 = mongo_db["T3_SMTPH_Zollfo_3"]

In [8]:
def get_soup(URL):
    r = requests.get(URL)
    demo=r.text
    return BeautifulSoup(demo,"html.parser") 

def get_pages(soup):
    pages = 1
    while soup.find_all(attrs = "next") != []:
        URL = "http://detail.zol.com.cn" + soup.find_all(attrs = "next")[0].attrs['href']
        soup = get_soup(URL)
        pages += 1
    return pages

def get_model_info(URL, brand):
    soup = get_soup(URL)
    pages = get_pages(soup)
    page = 1
    while (page <= pages):
        model_list = soup.find_all(name = "ul", attrs = {"id":"J_PicMode"})[0].find_all(name = "h3")
        for i in range(len(model_list)):
            model = model_list[i].find_previous_sibling(name = "a").img.attrs["alt"]
            model_price = model_list[i].find_next_sibling(attrs = {"class":"price-row"}).find(attrs = {"class":"price-type"}).string
            url = "http://detail.zol.com.cn" + model_list[i].find_previous_sibling(name = "a").attrs['href']
            dict_2["brand"].append(brand)
            dict_2["title"].append(model)
            dict_2["price"].append(model_price)
            dict_2["url"].append(url)
        if (page<=pages-1):
            URL = "http://detail.zol.com.cn" + soup.find_all(attrs = "next")[0].attrs['href']
            soup = get_soup(URL)
        page += 1

URL = "http://top.zol.com.cn/compositor/57/manu_attention.html"
soup = get_soup(URL)
list1 = soup.find_all(attrs = "name")
dict_1 = {"brand":[], "url":[]}
for i in range(len(list1)):  #len(list1)
    dict_1["brand"].append(list1[i].string)
    dict_1["url"].append("http:"+list1[i].attrs['href'])
df_1 = pd.DataFrame(dict_1)
mongo_coll_1.remove()
mongo_coll_1.insert(json.loads(df_1.T.to_json()).values())


dict_2 = {"brand":[], "title":[], "price":[], "url":[]}
for i in range(len(df_1)):
    get_model_info(df_1["url"][i], df_1["brand"][i])
df_2 = pd.DataFrame(dict_2)
df_2['index_zol'] = [re.findall('(?<=index)(.*)(?=\.shtml)', l)[0] for l in list(df_2['url'])]
df_2['create_time'] = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) 
df_2['update_time'] = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) 
df_2.insert(loc=0, column='hist_date', value=time.strftime("%Y-%m-%d", time.localtime()) )
mongo_coll_2.remove()
mongo_coll_2.insert(json.loads(df_2.T.to_json()).values())

<ipython-input-8-4fbadf22169f>:41: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  mongo_coll_1.remove()
<ipython-input-8-4fbadf22169f>:42: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  mongo_coll_1.insert(json.loads(df_1.T.to_json()).values())
<ipython-input-8-4fbadf22169f>:53: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  mongo_coll_2.remove()
<ipython-input-8-4fbadf22169f>:54: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  mongo_coll_2.insert(json.loads(df_2.T.to_json()).values())


[ObjectId('608ea36f2849d2250f1c68a9'),
 ObjectId('608ea36f2849d2250f1c68aa'),
 ObjectId('608ea36f2849d2250f1c68ab'),
 ObjectId('608ea36f2849d2250f1c68ac'),
 ObjectId('608ea36f2849d2250f1c68ad'),
 ObjectId('608ea36f2849d2250f1c68ae'),
 ObjectId('608ea36f2849d2250f1c68af'),
 ObjectId('608ea36f2849d2250f1c68b0'),
 ObjectId('608ea36f2849d2250f1c68b1'),
 ObjectId('608ea36f2849d2250f1c68b2'),
 ObjectId('608ea36f2849d2250f1c68b3'),
 ObjectId('608ea36f2849d2250f1c68b4'),
 ObjectId('608ea36f2849d2250f1c68b5'),
 ObjectId('608ea36f2849d2250f1c68b6'),
 ObjectId('608ea36f2849d2250f1c68b7'),
 ObjectId('608ea36f2849d2250f1c68b8'),
 ObjectId('608ea36f2849d2250f1c68b9'),
 ObjectId('608ea36f2849d2250f1c68ba'),
 ObjectId('608ea36f2849d2250f1c68bb'),
 ObjectId('608ea36f2849d2250f1c68bc'),
 ObjectId('608ea36f2849d2250f1c68bd'),
 ObjectId('608ea36f2849d2250f1c68be'),
 ObjectId('608ea36f2849d2250f1c68bf'),
 ObjectId('608ea36f2849d2250f1c68c0'),
 ObjectId('608ea36f2849d2250f1c68c1'),
 ObjectId('608ea36f2849d2

In [9]:
attrs={'screen_size':'主屏尺寸',
       'release_date': '上市日期',
       'physical_type': '手机类型',
       'camera_type1': '摄像头类型',
       'camera_type2': '摄像头总数',
       'protection': '三防功能',
       'back_cam': '后置摄像头',
       'front_cam': '前置摄像头',
       'cam_style': '摄像头特色',
       'ppi': '屏幕像素密度',
       'screen_material': '主屏材质',
       'screen_perc': '屏幕占比',
       'screen_others': '其他屏幕参数',
       'cpu_brand': 'CPU型号',
       'ram': 'RAM容量',
       'rom': 'ROM容量',
       'battery_vol': '电池容量',
       'battery_charge': '电池充电',
       'material': '机身材质',
       'other_function': '其他功能参数',
       'finger_print': '解锁方式',
       'sensor': '感应器类型',
       'interface': '机身接口',
       'device_size': '手机尺寸',
       'G5': '5G网络',
       'OS': '操作系统',
       'screen_fresh_rate': '屏幕刷新率'}

def generate_dict():
    dict_3 = {"index_zol":[],"back_cam":[], "battery_charge":[], "battery_vol":[], "camera_type1":[], "camera_type2":[],\
          "cpu_brand":[],"front_cam":[], "link":[], "material":[], "other_function":[], "physical_type":[], "ppi":[], \
           "protection":[], "ram":[], "release_date":[], "rom":[], "screen_material":[], "screen_others":[],\
          "screen_perc":[], "screen_size":[], "alias":[], "finger_print":[], "sensor":[], "interface":[],"cam_style":[],\
          "device_size":[],"G5":[],"OS":[],"screen_fresh_rate":[]}
    return dict_3

def get_soup(URL):
    r = requests.get(URL)
    demo=r.text
    return BeautifulSoup(demo,"html.parser") 

def get_df(x):
    tmp = list(x)
    return pd.DataFrame(tmp)

def get_model_info(url, dict_3):
    soup = get_soup(url)  
    if soup.find(attrs = "product-model__alias") is None:
        dict_3["alias"].append(None)
    else:
        dict_3["alias"].append(soup.find(attrs = "product-model__alias").string)
    dict_3["link"].append("http://detail.zol.com.cn"+soup.find(attrs = "nav__list clearfix").a.attrs["href"])
    dict_3["index_zol"].append(re.findall('(?<=index)(.*)(?=\.shtml)', url)[0])
    
    link = dict_3["link"][-1]
    soup = get_soup(link)
    html = etree.HTML(str(soup))
    for k in attrs.keys():
        attr = attrs[k]
        attr_value = html.xpath("string(//table//span[contains(.,'"+attr+"')]/../following-sibling::td/span)")
        if attr_value is None or attr_value == '':
            attr_value = html.xpath("string(//table//a[contains(.,'"+attr+"')]/../following-sibling::td/span)")
        if attr_value is None:
            dict_3[k].append('')
        else:
            dict_3[k].append(attr_value)

tmp_1 = mongo_coll_1.find({},{"brand":1})
brand_df = get_df(tmp_1)

tmp_3 = mongo_coll_3.find({},{"index_zol":1})
zols = get_df(tmp_3)

if len(zols) == 0:
    zols = []
else:
    zols = zols["index_zol"].values

for brand in brand_df["brand"].values:
    dict_3 = generate_dict()
    tmp_2 = mongo_coll_2.find({"brand": brand}, {"url":1, "index_zol":1})
    df_2 = get_df(tmp_2)
    for i in range(len(df_2)):
        if not df_2["index_zol"][i] in zols:
            get_model_info(df_2["url"][i], dict_3)
    df_tmp = pd.DataFrame(dict_3)
    if len(df_tmp) != 0:
        mongo_coll_3.insert(json.loads(df_tmp.T.to_json()).values())

<ipython-input-9-fd651f877b96>:88: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  mongo_coll_3.insert(json.loads(df_tmp.T.to_json()).values())
